In [49]:
import qiime2
import qiime2.plugins.feature_table
import pandas as pd
import itertools
import scipy.stats
import numpy as np

In [2]:
md = qiime2.Metadata.load('../sample-metadata-temp.tsv')
master_table = qiime2.Artifact.load('../100_otus/table-filtered.qza')

# number of samples a feature must be observed in to be included in these analyses
min_samples_fraction = 0.33

In [3]:
qiime2.Artifact.peek('../100_otus/table.qza')

ResultMetadata(uuid='455f7355-247e-4d4d-b506-2b986201c4c1', type='FeatureTable[Frequency]', format='BIOMV210DirFmt')

In [4]:
ft = qiime2.Artifact.load('../100_otus/table.qza')
ft = qiime2.plugins.feature_table.actions.filter_samples(ft, metadata=md, where="IncludedIn2017Analysis='Yes'").filtered_table
min_samples = int(ft.view(pd.DataFrame).shape[0] * min_samples_fraction)
ft = qiime2.plugins.feature_table.actions.filter_features(ft, min_samples=min_samples).filtered_table

In [5]:
ft_summary = qiime2.plugins.feature_table.actions.summarize(ft, sample_metadata=md).visualization

/Users/gregcaporaso/miniconda3/envs/qiime2-2018.6/lib/python3.5/site-packages/matplotlib/axes/_axes.py:6462: UserWarning: The 'normed' kwarg is deprecated, and has been replaced by the 'density' kwarg.
  warnings.warn("The 'normed' kwarg is deprecated, and has been "
/Users/gregcaporaso/miniconda3/envs/qiime2-2018.6/lib/python3.5/site-packages/matplotlib/axes/_axes.py:6462: UserWarning: The 'normed' kwarg is deprecated, and has been replaced by the 'density' kwarg.
  warnings.warn("The 'normed' kwarg is deprecated, and has been "


In [6]:
ft_summary

<visualization: Visualization uuid: c83239f0-aa5b-498f-83ea-33d3ec9df3a7>

In [7]:
# for this analysis we need to keep all samples, so even sampling depth is set to the minimum sample frequency. 
# Samples with extremely low total frequencies have already been filtered from this table (see the 
# IncludedIn2017Analysis metadata category).
even_sampling_depth = int(ft.view(pd.DataFrame).sum(axis=1).min())

In [8]:
# Since this step is non-deterministic, I comment it out so it can't accidentally be re-run. 
# ft_rare = qiime2.plugins.feature_table.actions.rarefy(ft, even_sampling_depth).rarefied_table

In [9]:
print(ft_rare.view(pd.DataFrame).shape)

(802, 106)


In [10]:
import os
import os.path
os.getcwd()

'/Users/gregcaporaso/Google Drive/data-analysis/2017.06-udca-manuscript-analyses/network-analysis'

Compute correlations with Spearman and Pearson, and generate commands to compute SparCC correlations. SparCC is Python 2.6 software, so needs to run in its own environment.

In [106]:
treatments = ['UDCA', 'placebo'] # treatmentgroup
visits = ['pre', 'post'] # visit
sparcc_command_template = (
        "python /Users/gregcaporaso/code/crc-udca1/network-analysis/run-sparcc.py "
        " \"%s\" 1000 \"%s\"")
cwd = os.getcwd()
sparcc_cmds = []

for t, v in itertools.product(treatments, visits):
    output_dir = os.path.join(cwd, '%s-%s' % (t, v))
    # SparCC takes a long time to run, so this should fail if the 
    # output directory already exists so those results aren't overwritten.
    os.makedirs(output_dir, exist_ok=False)
    temp_ft = qiime2.plugins.feature_table.actions.filter_samples(ft_rare, 
                                                                  metadata=md, 
                                                                  where="treatmentgroup='%s' AND visit='%s'" % (t, v)).filtered_table
    temp_ft.save(os.path.join(output_dir, 'table.qza'))
    
    df = temp_ft.view(pd.DataFrame)
    table_fn = "sparcc-table.tsv"
    table_fp = os.path.join(output_dir, table_fn)
    sparcc_output_dn = "sparcc"
    sparcc_output_dp = os.path.join(output_dir, sparcc_output_dn)
    
    df.T.to_csv(table_fp, sep='\t', index_label='OTU_ID')
    sparc_cmd = sparcc_command_template % (table_fp, sparcc_output_dp)
    sparcc_cmds.append(sparc_cmd)
    
    spearman_rho, spearman_p = scipy.stats.spearmanr(df)
    pd.DataFrame(spearman_rho, index=df.columns, columns=df.columns).to_csv(
            os.path.join(output_dir, "spearman_rho.tsv"), sep='\t', index_label='OTU_ID')
    pd.DataFrame(spearman_p, index=df.columns, columns=df.columns).to_csv(
            os.path.join(output_dir, "spearman_p.tsv"), sep='\t', index_label='OTU_ID')
    
    # scipy.stats.pearsonr has a different interface than scipy.stats.spearmanr :(
    pearson_r = []
    pearson_p = []
    for _, r1 in df.T.iterrows():
        pearson_r_row = []
        pearson_p_row = []
        for _, r2 in df.T.iterrows():
            r, p = scipy.stats.pearsonr(r1, r2)
            pearson_r_row.append(r)
            pearson_p_row.append(p)
        pearson_r.append(pearson_r_row)
        pearson_p.append(pearson_p_row)
    pd.DataFrame(pearson_r, index=df.columns, columns=df.columns).to_csv(
            os.path.join(output_dir, "pearson_r.tsv"), sep='\t', index_label='OTU_ID')
    pd.DataFrame(pearson_p, index=df.columns, columns=df.columns).to_csv(
            os.path.join(output_dir, "pearson_p.tsv"), sep='\t', index_label='OTU_ID')

print(' && '.join(sparcc_cmds))


python /Users/gregcaporaso/code/crc-udca1/network-analysis/run-sparcc.py  "/Users/gregcaporaso/Google Drive/data-analysis/2017.06-udca-manuscript-analyses/network-analysis/UDCA-pre/sparcc-table.tsv" 1000 "/Users/gregcaporaso/Google Drive/data-analysis/2017.06-udca-manuscript-analyses/network-analysis/UDCA-pre/sparcc" && python /Users/gregcaporaso/code/crc-udca1/network-analysis/run-sparcc.py  "/Users/gregcaporaso/Google Drive/data-analysis/2017.06-udca-manuscript-analyses/network-analysis/UDCA-post/sparcc-table.tsv" 1000 "/Users/gregcaporaso/Google Drive/data-analysis/2017.06-udca-manuscript-analyses/network-analysis/UDCA-post/sparcc" && python /Users/gregcaporaso/code/crc-udca1/network-analysis/run-sparcc.py  "/Users/gregcaporaso/Google Drive/data-analysis/2017.06-udca-manuscript-analyses/network-analysis/placebo-pre/sparcc-table.tsv" 1000 "/Users/gregcaporaso/Google Drive/data-analysis/2017.06-udca-manuscript-analyses/network-analysis/placebo-pre/sparcc" && python /Users/gregcaporaso

In [116]:
alpha = 0.001
summary = []
summary_columns = ['treatmentgroup', 'visit', 'Spearman significant', 'Pearson significant', 'SparCC significant', 
                   'Ensemble significant', 'Same sign', 'Reported interactions']

for t, v in itertools.product(treatments, visits):
    row_summary = [t, v]
    data_dir = os.path.join(cwd, '%s-%s' % (t, v))
    
    spearman_rho_df = pd.read_csv(os.path.join(data_dir, "spearman_rho.tsv"), sep='\t').set_index('OTU_ID')
    spearman_p_df = pd.read_csv(os.path.join(data_dir, "spearman_p.tsv"), sep='\t').set_index('OTU_ID')
    row_summary.append(np.count_nonzero(spearman_p_df <= alpha))
    
    pearson_r_df = pd.read_csv(os.path.join(data_dir, "pearson_r.tsv"), sep='\t').set_index('OTU_ID')
    pearson_p_df = pd.read_csv(os.path.join(data_dir, "pearson_p.tsv"), sep='\t').set_index('OTU_ID')
    row_summary.append(np.count_nonzero(pearson_p_df <= alpha))
    
    sparcc_r_df = pd.read_csv(os.path.join(data_dir, 'sparcc', 'corr.out'), sep='\t').set_index('OTU_ID')
    sparcc_p_df = pd.read_csv(os.path.join(data_dir, 'sparcc', 'p-value.out'), sep='\t').set_index('OTU_ID')
    row_summary.append(np.count_nonzero(sparcc_p_df <= alpha))
    
    significance_df = (pearson_p_df <= alpha) & (spearman_p_df <= alpha) & (sparcc_p_df <= alpha)
    same_sign_df = (np.sign(pearson_r_df) == np.sign(spearman_rho_df)) == np.sign(sparcc_r_df)
    report_interaction_df = significance_df & same_sign_df
    
    row_summary.append(np.count_nonzero(significance_df))
    row_summary.append(np.count_nonzero(same_sign_df))
    row_summary.append(np.count_nonzero(report_interaction_df))
    
    significance_df.to_csv(os.path.join(data_dir, 'ensemble-significance.tsv'),
                              sep='\t', index_label='OTU_ID')
    same_sign_df.to_csv(os.path.join(data_dir, 'ensemble-same-sign.tsv'),
                              sep='\t', index_label='OTU_ID')
    report_interaction_df.to_csv(os.path.join(data_dir, 'report-interaction.tsv'),
                              sep='\t', index_label='OTU_ID')
    summary.append(row_summary)

summary_df = pd.DataFrame(summary, columns=summary_columns)

In [117]:
summary_df

,treatmentgroup,visit,Spearman significant,Pearson significant,SparCC significant,Ensemble significant,Same sign,Reported interactions
0,UDCA,pre,1122,612,966,176,3792,174
1,UDCA,post,1260,586,978,222,3908,210
2,placebo,pre,1112,634,850,200,3794,192
3,placebo,post,884,544,716,152,3950,148
